# Introduction
This is a template to create, apply and validate a machine learning model for land use classificatin. The model is trained and valided with seperate shapefiles. The output include:

1. a statistic summary of the input training data for the model (export as .csv file).
2. a trained machine learning model. In this example a random forest model is used (export as a .pkl file). 
3. a classified image using the trained model (export as .tif). 
4. validating the classified image, generating an accuracy matrix and export as .csv file. 

# Using this template 

Set the variables in the first cell to the following:
```python
pyeo_path = "The folder that you cloned pyeo into"
training_tif = "The path to the stacked image you used for your training data"
training_shp = "The path to the shapefile you made that contains a field with whole number (intergers) class identifiers"
training_field_name = "The name of the field in training_shp containing class identifiers. CODE, by default."
out_dir = "A directory that the model and related files will be put in"
validation_shp = "The shapefile containing the valiation data."
validation_field_name = "The name of the field in validation_shp containing class identifers. CODE by default."
```

Now run each cell.

In [ ]:
pyeo_path = "/home/ecci/pyeo"
training_tif = "xxxx.tif "
training_shp = "xxxxx.shp"
training_field_name = 'CODE'
out_dir = "Output/template3_model_building/"

validation_shp = "xxx.shp"
validation_field_name = 'CODE'

tobe_classified_tif = training_tif

In [2]:
import os,sys
sys.path.append(r"../templates/")
import PYEO_model
sys.path.append(pyeo_path)
import pyeo.core as pyeo
import numpy as np
import csv

/opt/tljh/user/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ModuleNotFoundError: No module named 'pyeo'

In [ ]:
#set output folder

out_model = os.path.join(out_dir,'model.pkl')
out_training_summary = os.path.join(out_dir, 'training_summary.csv')
out_classified = os.path.join(out_dir, "2018_classified.tif")

In [ ]:
features, classes = pyeo.get_training_data(training_tif, training_shp, attribute=training_field_name)

model, scores = PYEO_model.train_model_rf(features,classes)
pyeo.save_model(model, out_model) 
print("Cross-validation scores: {}".format(scores))

In [ ]:
# Explore training dataset by export the siganiture of each training pixel and summarise the statistics of each categoriy  
sigs = np.vstack((classes, features.T))
out_sig= os.path.join(out_dir, 'sig.csv')
with open(out_sig , 'w', newline='') as outsig:
    writer = csv.writer(outsig)
    writer.writerows(sigs.T)
# generate statistic summary for each class:
PYEO_model.summarise_training(in_classes= classes,out_csv= out_training_summary, sumarise_type = 'count')

In [ ]:
#classify image
pyeo.classify_image(image_path= tobe_classified_tif, model_path=out_model, class_out_path= out_classified,num_chunks =10)#need to chagne num_chunks


In [ ]:
PYEO_model.show_satellite_image(out_classified)

In [ ]:
# validate the map and generate accuracy matrix 
PYEO_model.validate_classes(inRaster=out_classified, shpdir=validation_shp , field_name=validation_field_name, 
                            out_fldr=out_dir, nodata=nodata)


